# A. Data augmentation

## Prompt an LLM to generate essays, so that you balance the data (use both prompts provided by the challenge).

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("train_essays.csv")
train_df

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [3]:
train_prompts_df = pd.read_csv("train_prompts.csv")
train_prompts_df

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [4]:
print(train_prompts_df.iloc[0,2])

Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.


In [5]:
print(train_prompts_df.iloc[1,2])

Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Use the information from the texts in your essay. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.


In [6]:
prompt1 = "Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay.Write up to 600 words."

In [7]:
prompt2 = "Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.Write up to 600 words."

In [8]:
from openai import OpenAI
import json
essays_list_prompt1 = []
client = OpenAI(api_key = "")

for i in range(700):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt1}
      ]
    )
    text = response.choices[0].message.content
    essays_list_prompt1.append(text)

In [9]:
essays_list_prompt2 = []
for i in range(700):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt2}
      ]
    )
    text = response.choices[0].message.content
    essays_list_prompt2.append(text)

In [37]:
def remove_punctuation_except_periods(text):
    return re.sub(r'[^\w\s.]', '', text)

In [39]:
def remove_specific_words(text):
    words_to_remove = ['essay', 'body', 'introduction','conclusion','title','response',' \n'] + [f'paragraph{x}' for x in range(10)]  # Adjust the range as needed
    pattern = '\\b(?:' + '|'.join(map(re.escape, words_to_remove)) + ')\\b'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

In [40]:
def clean_text(text):
    text = remove_punctuation_except_periods(text)
    text = remove_specific_words(text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    return text

In [13]:
train_df.drop(axis=1,columns=['id'],inplace=True)

In [14]:
for i in range(700):
    new_row1 = {'prompt_id':0, 'text':essays_list_prompt1[i], 'generated':1}
    train_df = train_df.append(new_row1, ignore_index=True)
    new_row2 = {'prompt_id':1, 'text':essays_list_prompt2[i], 'generated':1}
    train_df = train_df.append(new_row2, ignore_index=True)
train_df

C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row1, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row2, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row1, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.appe

C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row1, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row2, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row1, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.appe

C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row2, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row1, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row2, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.appe

C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row2, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row1, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(new_row2, ignore_index=True)
C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\2931818159.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.appe

,prompt_id,text,generated
0,0,Cars. Cars have been around since they became ...,0
1,0,Transportation is a large necessity in most co...,0
2,0,"""America's love affair with it's vehicles seem...",0
3,0,How often do you ride in a car? Do you drive a...,0
4,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...
2773,1,"{\n ""letter_to_state_senator"": {\n ""recipi...",1
2774,0,"\n{\n ""response"": {\n ""essay"": {\n ""i...",1
2775,1,"\n{\n ""letter"": {\n ""recipient"": ""State Se...",1
2776,0,"\n{""response"": ""Due to the character limit, I ...",1


In [41]:
import re
train_df['text'] = train_df['text'].apply(clean_text)

C:\Users\vassi\AppData\Local\Temp\ipykernel_20508\3344948122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text'] = train_df['text'].apply(clean_text)


In [32]:
mask = train_df['text'].str.contains('sorry', case=False) 
train_df = train_df[~mask]

In [33]:
train_df

,prompt_id,text,generated
0,0,Cars. Cars have been around since they became ...,0
1,0,Transportation is a large necessity in most co...,0
2,0,Americas love affair with its vehicles seems t...,0
3,0,How often do you ride in a car Do you drive a ...,0
4,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...
2773,1,letter_to_state_senator \n recipient Senator L...,1
2774,0,response \n \n Limiting car usage offers numer...,1
2775,1,letter \n recipient State Senator\n address St...,1
2776,0,response Due to the character limit I cannot p...,1


In [45]:
train_df.to_csv("train_df2.csv")

In [44]:
train_df[(train_df['generated']==1)&(train_df['prompt_id']==0)]

,prompt_id,text,generated
1262,0,Limiting car use causes pollution increases co...,1
1378,0,The Benefits of Limiting Car Usage for Our Com...,1
1380,0,Limiting car usage can have numerous advantage...,1
1382,0,The Advantages of Limiting Car Usage\n The rap...,1
1386,0,The overreliance on cars in our society has le...,1
...,...,...,...
2768,0,The Advantages of Limiting Car Usage for a Bet...,1
2770,0,As our cities grapple with traffic congestion ...,1
2772,0,As our society faces increasing concerns about...,1
2774,0,Limiting car usage offers numerous advantages ...,1
